In [ ]:
import xarray as xr
import numpy as np
import os

from sloppy.distrib import compute_block

import cartopy as cart
import cartopy.feature as cfeature
import cmocean as cmo
import matplotlib.pyplot as plt
import matplotlib.colorbar as cbar
import matplotlib.colors
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def proj_xy(lon, lat, PROJSTRING):
    """ """
    from pyproj import CRS, Transformer

    # create the coordinate reference system
    crs = CRS.from_proj4(PROJSTRING)
    # create the projection from lon/lat to x/y
    proj = Transformer.from_crs(crs.geodetic_crs, crs)
    # compute the lon/lat
    xx, yy = proj.transform(lon, lat, direction="FORWARD")
    return xx, yy

In [ ]:
def pltgrd(x, y, color, subsamp):
   nj,ni = x.shape
   for j in range(0,nj,subsamp):
      plt.plot(x[j,:],     y[j,:], color, transform=cart.crs.PlateCarree())
      plt.plot(x[j,:]+360, y[j,:], color, transform=cart.crs.PlateCarree())
   for i in range(0,ni,subsamp):
      plt.plot(x[:,i],     y[:,i], color, transform=cart.crs.PlateCarree())

## Path to data files

In [ ]:
FIXTURE_DIR = "sloppy/tests/data/"
print(FIXTURE_DIR)

## Data:
- mosaic (hgrid)
- topography (gebco)

In [ ]:
gridfile = "mom6_grid_global_coarse.nc"
topofile = "gebco_reduced.nc"

dstopo = xr.open_dataset(FIXTURE_DIR + topofile, decode_times=False)
dsgrid = xr.open_dataset(FIXTURE_DIR + gridfile, decode_times=False)

print(dsgrid)

## Plot grid

In [ ]:
xg = dsgrid["x"][0::2, 0::2].values
yg = dsgrid["y"][0::2, 0::2].values

plt.figure(num=1, figsize=(10,10))
ax = plt.subplot(111, projection=cart.crs.PlateCarree(central_longitude=-160))
ax.coastlines()

pp = [[0,dsgrid["x"].shape[0],'k']]

ss = 4 # skip `ss/2` points
for p in pp:
    js = slice(p[0],p[1])
    pltgrd(xg[js,:], yg[js,:], p[2], ss)
plt.title('plot skip= %.1f'%(0.5*ss))

In [ ]:
lontopo, lattopo = np.meshgrid(dstopo["lon"].values, dstopo["lat"].values)

assert len(lontopo.shape) == 2
assert len(lattopo.shape) == 2

In [ ]:
## compute elevation
## elev: [cell estimated depth, minimum depth in cell, maximum depth in cell, residual of plane fit, number of source points in cell]
elev = compute_block(
        dsgrid["x"][0::2, 0::2].values,
        dsgrid["y"][0::2, 0::2].values,
        dstopo["elevation"].values,
        lontopo,
        lattopo,
        is_stereo=False,
        is_carth=False,
        PROJSTRING=None,
        residual=True,
)

assert elev[0, :, :].max() <= 99999.0
assert elev[0, :, :].min() >= -99999.0

## Plot elevation

In [ ]:
vmin,vmax,ci,cmap = 0,6000.,500.,cmo.cm.topo_r
cilev = np.arange(vmin-ci,vmax+ci*2,ci)
norm = matplotlib.colors.BoundaryNorm(boundaries=cilev, ncolors=cmap.N)
#

plt.figure(num=1, figsize=(10,10))
#ax = plt.subplot(111, projection=cart.crs.Robinson(central_longitude=-160))
ax = plt.subplot(111, projection=cart.crs.PlateCarree(central_longitude=-160))
im = ax.pcolormesh(xg, yg, -elev[0,:,:],\
                   transform=cart.crs.PlateCarree(), cmap=cmap, vmin=vmin, vmax=vmax)
ax.coastlines(color='r')
ax.add_feature(cfeature.LAND)

ax = plt.gcf().add_axes((.91,.32,.02,.35))
cb = cbar.ColorbarBase(ax=ax, cmap=cmap, norm=norm, boundaries=cilev,\
                       orientation='vertical', extend='both')
cb.ax.set_ylabel('cell estimated depth (m)')

## Recompute using lat/lon 1-d arrays of topography instead of 2-d (as above)

In [ ]:
# compute elevation
elev = compute_block(
        dsgrid["x"][0::2, 0::2].values,
        dsgrid["y"][0::2, 0::2].values,
        dstopo["elevation"].values,
        dstopo["lon"].values,
        dstopo["lat"].values,
        is_stereo=False,
        is_carth=False,
        PROJSTRING=None,
        residual=True,
)

assert elev[0, :, :].max() <= 99999.0
assert elev[0, :, :].min() >= -99999.0

## Re-plot elevation

In [ ]:
vmin,vmax,ci,cmap = 0,6000.,500.,cmo.cm.topo_r
cilev = np.arange(vmin-ci,vmax+ci*2,ci)
norm = matplotlib.colors.BoundaryNorm(boundaries=cilev, ncolors=cmap.N)
#

plt.figure(num=1, figsize=(10,10))
ax = plt.subplot(111, projection=cart.crs.PlateCarree(central_longitude=-160))

im = ax.pcolormesh(xg, yg, -elev[0,:,:],\
                   transform=cart.crs.PlateCarree(), cmap=cmap, vmin=vmin, vmax=vmax)
ax.coastlines(color='r')
ax.add_feature(cfeature.LAND)

ax = plt.gcf().add_axes((.91,.32,.02,.35))
cb = cbar.ColorbarBase(ax=ax, cmap=cmap, norm=norm, boundaries=cilev,\
                       orientation='vertical', extend='both')
cb.ax.set_ylabel('cell estimated depth (m)')